<a href="https://colab.research.google.com/github/Stephen-sanchez1/sqlite_database_operations/blob/main/sqlite_database_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3
import matplotlib.pyplot as plt
import numpy as np

## 1. Data Exploration and Analysis:

## Loading Data

In [101]:
df_sbu = pd.read_csv('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/stonybrook/stonybrook.csv')

df_nyp = pd.read_json('https://raw.githubusercontent.com/hantswilliams/HHA_504_2023/main/WK3/data/nyp/133957095_NewYorkPresbyterianHospital_standardcharges.json')


In [102]:
df_sbu.head(10)


,Code,Description,Type,Package/Line_Level,Gross charge,Discounted cash price,De-identified min contracted rate,De-identified max contracted rate,Derived contracted rate,1199-Commercial other,...,Optum-Commercial other,Oxford-Commercial other,Oxford-Commercial HMO/POS,Tricare-Commercial other,United Healthcare-Commercial other,United Healthcare-Medicare Advantage HMO,United Healthcare-Commercial HMO/POS,United Healthcare-Medicaid HMO,United Healthcare-Commercial PPO/Open Access,Veteran Family-Commercial other
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, EACH ADDITIONAL...",Outpatient,Line,718.67,718.67,2155.50,2155.50,2155.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
2,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Package,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
3,10006,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,728.54,728.54,58.55,1761.25,909.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10009,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Line,2823.01,2823.01,446.28,6017.59,4219.31,4592.0,...,NaN,NaN,NaN,NaN,5127.00,NaN,NaN,NaN,NaN,NaN
5,10009,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Package,2823.01,2823.01,446.28,6017.59,4219.31,4592.0,...,NaN,NaN,NaN,NaN,5127.00,NaN,NaN,NaN,NaN,NaN
6,10010,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Line,588.00,588.00,2155.50,2155.50,2155.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10021,"FINE NEEDLE ASPIRATION BIOPSY, FIRST GROWTH",Outpatient,Line,1566.78,1566.78,275.46,2265.50,1270.48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10021,"FINE NEEDLE ASPIRATION BIOPSY, FIRST GROWTH",Outpatient,Package,1566.78,1566.78,275.46,2265.50,1270.48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10030,DRAINAGE OF FLUID COLLECTION IN SOFT TISSUE US...,Outpatient,Line,2082.05,2082.05,152.66,4101.00,1663.85,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
df_nyp.head(10)

,Code (CPT/DRG),Description,Rev Code,Inpatient/Outpatient,Gross Charges,Discounted Cash Price,Aetna,Cigna,Empire Blue Cross Blue Shield,Emblem Health,...,Consumer Health Network,Devon,Equian,First Health,Magnacare,Multiplan/Beechstreet/PHCS,QHM,Worldwide,Minimum Negotiated Charge,Maximum Negotiated Charge
0,96360,HC IV INFUSION HYDRATION INITIAL 31 MIN-1HR,0260,Inpatient/Outpatient,866.0,866.0,641.706,494.486,640.84,1388.424,...,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,393.5104,253.04825,1388.424000
1,96361,HC IV INFUSION HYDRATION FLUIDS ADDL HR,0260,Inpatient/Outpatient,358.0,358.0,265.278,204.418,264.92,1305.112,...,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,162.6752,51.90325,1990.891341
2,96365,HC IV INFUSION FOR THER/PROPH/DIAG INITIAL UP ...,0260,Inpatient/Outpatient,1397.0,1397.0,1035.177,797.687,1033.78,1475.508,...,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,634.7968,253.04825,1475.508000
3,96367,HC IV INF THER/PROPH/DIAG ADDL SEQ NEW DRUG UP...,0260,Inpatient/Outpatient,480.0,480.0,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,82.65075,1325.120000
4,96368,HC IV INFUSION FOR THER/PROPH/DIAG CONCURRENT,0260,Inpatient/Outpatient,480.0,480.0,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,156.98280,1325.120000
5,96369,HC NON-CHEMO SC INFUSION 1 HR,0260,Inpatient/Outpatient,780.0,780.0,577.98,445.38,577.2,1374.32,...,354.432,354.432,354.432,354.432,354.432,354.432,354.432,354.432,253.04825,1374.320000
6,96370,HC NON-CHEMO SC INFUSION EACH ADDL HR,0260,Inpatient/Outpatient,480.0,480.0,355.68,274.08,355.2,1325.12,...,218.112,218.112,218.112,218.112,218.112,218.112,218.112,218.112,51.90325,1990.891341
7,96375,HC THERAPEUTIC/PROPH/DIAG INJ IV PUSH ADDL SEQ,0260,Inpatient/Outpatient,602.0,602.0,446.082,343.742,445.48,1345.128,...,273.5488,273.5488,273.5488,273.5488,273.5488,273.5488,273.5488,273.5488,51.90325,1345.128000
8,96376,HC THERAPEUTIC/PROPH/DIAG INJ IV PUSH SAME,0260,Inpatient/Outpatient,357.0,357.0,264.537,203.847,264.18,1304.948,...,162.2208,162.2208,162.2208,162.2208,162.2208,162.2208,162.2208,162.2208,162.22080,1304.948000
9,96377,HC APPLICATION OF BODY INJECTOR,0260,Inpatient/Outpatient,510.0,510.0,377.91,291.21,377.4,1330.04,...,231.744,231.744,231.744,231.744,231.744,231.744,231.744,231.744,51.90325,1330.040000


## Basic Statistics

In [104]:
# Basic statistics for Stony Brook
print("SBU Descriptive Statistics")
print(df_sbu.describe())


SBU Descriptive Statistics
       Gross charge  Discounted cash price  De-identified min contracted rate  \
count  7.710000e+03           7.710000e+03                        6254.000000   
mean   2.274832e+04           2.274832e+04                        4090.693876   
std    7.470570e+04           7.470570e+04                       10576.213503   
min    3.000000e-02           3.000000e-02                           0.010000   
25%    1.065895e+03           1.065895e+03                          96.520000   
50%    4.745295e+03           4.745295e+03                        1108.860000   
75%    1.103881e+04           1.103881e+04                        4309.532500   
max    1.857952e+06           1.857952e+06                      327869.690000   

       De-identified max contracted rate  Derived contracted rate  \
count                       6.254000e+03              6254.000000   
mean                        1.003579e+04              6315.587725   
std                         2.915075

In [105]:
#Basic Staistics for New York Presbyterian
print("NYP Descriptive Statistics")
print(df_nyp.describe())

NYP Descriptive Statistics
       Gross Charges  Discounted Cash Price  Minimum Negotiated Charge  \
count   6.193000e+03           6.193000e+03                5912.000000   
mean    3.385988e+04           3.385988e+04                2249.918880   
std     1.062409e+05           1.062409e+05                6015.717204   
min     1.000000e+00           1.000000e+00                   0.454400   
25%     5.970000e+02           5.970000e+02                 122.228437   
50%     3.925000e+03           3.925000e+03                 431.142905   
75%     3.033600e+04           3.033600e+04                2107.000000   
max     2.538980e+06           2.538980e+06              204861.126000   

       Maximum Negotiated Charge  
count               5.912000e+03  
mean                2.390277e+04  
std                 7.136503e+04  
min                 1.608000e+00  
25%                 6.380010e+02  
50%                 4.086731e+03  
75%                 1.925640e+04  
max                 1.7134

In [106]:
print(df_sbu.columns)

Index(['Code', 'Description', 'Type', 'Package/Line_Level', 'Gross charge',
       'Discounted cash price', 'De-identified min contracted rate',
       'De-identified max contracted rate', 'Derived contracted rate',
       '1199-Commercial other', 'Aetna-Medicare Advantage HMO',
       'Aetna-Commercial HMO/POS', 'Aetna-Commercial PPO/Open Access',
       'Aetna-Commercial other', 'Empire Health-Commercial other',
       'Empire Health-Commercial PPO/Open Access',
       'BlueCross BlueShield-Commercial other',
       'Beacon Health-Commercial other', 'Carelon Health-Commercial other',
       'Cigna-Commercial PPO/Open Access', 'Cigna-Commercial other',
       'Cigna-Commercial HMO/POS', 'EH Facet-Commercial other',
       'EmblemHealth-Commercial PPO/Open Access',
       'EmblemHealth-Commercial other', 'EmblemHealth-Commercial HMO/POS',
       'EmblemHealth-Medicaid HMO', 'EmblemHealth-Medicare Advantage HMO',
       'Empire Health-Commercial HMO/POS',
       'Empire Health-Medicare 

In [107]:
# Frequency counts for SBU data
df_sbu['Gross charge'].value_counts()

8165.00     268
12090.00     70
13590.00     63
8965.00      47
513.00       13
           ... 
6453.95       1
3727.11       1
2284.63       1
18455.69      1
4580.00       1
Name: Gross charge, Length: 5871, dtype: int64

In [108]:
print(df_nyp.columns)

Index(['Code (CPT/DRG)', 'Description', 'Rev Code', 'Inpatient/Outpatient',
       'Gross Charges', 'Discounted Cash Price', 'Aetna', 'Cigna',
       'Empire Blue Cross Blue Shield', 'Emblem Health', 'United Health Group',
       'Aetna Medicare', 'AgeWell Medicare', 'Emblem Medicare',
       'Empire Medicare', 'Fidelis Medicare', 'Healthfirst Medicare',
       'UHC Community Plan/United Medicare', 'VNS Medicare',
       'WellCare Medicare', '1199', 'Affinity Molina Essential',
       'Affinity Molina Medicaid/CHP', 'Amida Care Medicaid',
       'Emblem Medicaid/CHP', 'Empire Healthplus Essential',
       'Empire Healthplus Exchange', 'Empire Healthplus Medicaid/CHP',
       'Fidelis Essential/Exchange', 'Fidelis Medicaid/CHP',
       'Healthfirst Essential/Exchange', 'Healthfirst Medicaid/CHP',
       'MVP Medicaid/CHP', 'MVP Essential', 'United Community Plan Essential',
       'United Community Plan Medicaid', 'VNS Medicaid',
       'Consumer Health Network', 'Devon', 'Equian', 'Fir

In [109]:
# Frequency counts for NYP data
df_nyp['Gross Charges'].value_counts()

122.000000      32
3337.000000     29
2553.000000     27
618.000000      26
116.000000      23
                ..
47682.000000     1
26559.000000     1
39648.000000     1
49748.000000     1
77081.639496     1
Name: Gross Charges, Length: 4101, dtype: int64

In [110]:
# Check for missing values
df_sbu.isnull().sum()

Code                                               0
Description                                        0
Type                                             783
Package/Line_Level                               783
Gross charge                                       0
Discounted cash price                              0
De-identified min contracted rate               1456
De-identified max contracted rate               1456
Derived contracted rate                         1456
1199-Commercial other                           6696
Aetna-Medicare Advantage HMO                    6604
Aetna-Commercial HMO/POS                        5883
Aetna-Commercial PPO/Open Access                6098
Aetna-Commercial other                          7582
Empire Health-Commercial other                  4650
Empire Health-Commercial PPO/Open Access        5308
BlueCross BlueShield-Commercial other           6456
Beacon Health-Commercial other                  7707
Carelon Health-Commercial other               

In [111]:
# Check for missing values
df_nyp.isnull().sum()

Code (CPT/DRG)                          0
Description                             0
Rev Code                                0
Inpatient/Outpatient                    0
Gross Charges                           0
Discounted Cash Price                   0
Aetna                                   0
Cigna                                   0
Empire Blue Cross Blue Shield           0
Emblem Health                           0
United Health Group                     0
Aetna Medicare                          0
AgeWell Medicare                        0
Emblem Medicare                         0
Empire Medicare                         0
Fidelis Medicare                        0
Healthfirst Medicare                    0
UHC Community Plan/United Medicare      0
VNS Medicare                            0
WellCare Medicare                       0
1199                                    0
Affinity Molina Essential               0
Affinity Molina Medicaid/CHP            0
Amida Care Medicaid               

In [112]:
import re
def clean_column_names(df_sbu):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    df_sbu.columns = [clean_name(col) for col in df_sbu.columns]
    return df_sbu

# Apply the clean_value function to all columns
df_sbu = clean_column_names(df_sbu)



In [113]:
def clean_column_names(df_nyp):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    df_nyp.columns = [clean_name(col) for col in df_nyp.columns]
    return df_sbu

# Apply the clean_value function to all columns
df_nyp = clean_column_names(df_nyp)


In [114]:
print("Missing Values in Dataset 1:")
print(df_sbu.isnull().sum())

Missing Values in Dataset 1:
code                                          0
description                                   0
type                                        783
packagelinelevel                            783
grosscharge                                   0
discountedcashprice                           0
deidentifiedmincontractedrate              1456
deidentifiedmaxcontractedrate              1456
derivedcontractedrate                      1456
1199commercialother                        6696
aetnamedicareadvantagehmo                  6604
aetnacommercialhmopos                      5883
aetnacommercialppoopenaccess               6098
aetnacommercialother                       7582
empirehealthcommercialother                4650
empirehealthcommercialppoopenaccess        5308
bluecrossblueshieldcommercialother         6456
beaconhealthcommercialother                7707
carelonhealthcommercialother               7707
cignacommercialppoopenaccess               5744
cignacommer

In [115]:
print("Missing Values in Dataset 2:")
print(df_nyp.isnull().sum())

Missing Values in Dataset 2:
code                                          0
description                                   0
type                                        783
packagelinelevel                            783
grosscharge                                   0
discountedcashprice                           0
deidentifiedmincontractedrate              1456
deidentifiedmaxcontractedrate              1456
derivedcontractedrate                      1456
1199commercialother                        6696
aetnamedicareadvantagehmo                  6604
aetnacommercialhmopos                      5883
aetnacommercialppoopenaccess               6098
aetnacommercialother                       7582
empirehealthcommercialother                4650
empirehealthcommercialppoopenaccess        5308
bluecrossblueshieldcommercialother         6456
beaconhealthcommercialother                7707
carelonhealthcommercialother               7707
cignacommercialppoopenaccess               5744
cignacommer

## 2. SQLite Database Operations

---



In [143]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect("health.db")
c = conn.cursor()

#create table
c.execute('''
    CREATE TABLE IF NOT EXISTS Stony_Brook (
        patient_name TEXT,
        diagnosis TEXT,
        insurance_accepted TEXT,
        hospital_charge REAL,
        patient_cost REAL
    )
''')

conn.commit()


In [144]:
# Inserting data into Stony_Brook
sql_query = """

INSERT INTO Stony_Brook (
  'patient_name',
  'diagnosis',
  'insurance_accepted',
  'hospital_charge',
  'patient_cost'
  )
  values (
    'John Doe',
    'Appendicitis',
    'Yes',
    12000.50,
    2000.00
  ),
  (
    'Jane Smith',
    'ACL Tear',
    'No',
    8000.75,
    8000.75
  );

"""
print(sql_query)




INSERT INTO Stony_Brook (
  'patient_name',
  'diagnosis',
  'insurance_accepted',
  'hospital_charge',
  'patient_cost'
  )
  values (
    'John Doe',
    'Appendicitis',
    'Yes',
    12000.50,
    2000.00
  ),
  (
    'Jane Smith',
    'ACL Tear',
    'No',
    8000.75,
    8000.75
  );




In [149]:
conn.commit()

In [148]:
# Create engine to connect to our sqlite DB
engine = create_engine('sqlite:///health.db')

In [152]:
# Check rows in Table
query = """

  SELECT *
  from Stony_Brook;

  """

c.execute(query)
print(c.fetchall())

[(1, 'John Doe', 'Appendicitis', 'Yes', 12000.5, 2000.0), (2, 'Jane Smith', 'ACL Tear', 'No', 8000.75, 8000.75), (3, 'John Doe', 'Appendicitis', 'Yes', 12000.5, 2000.0), (4, 'Jane Smith', 'ACL Tear', 'No', 8000.75, 8000.75)]


In [153]:
# Table display
Stony_Brook = pd.read_sql(query, conn)
Stony_Brook

,id,patient_name,diagnosis,insurance_accepted,hospital_charge,patient_cost
0,1,John Doe,Appendicitis,Yes,12000.50,2000.00
1,2,Jane Smith,ACL Tear,No,8000.75,8000.75
2,3,John Doe,Appendicitis,Yes,12000.50,2000.00
3,4,Jane Smith,ACL Tear,No,8000.75,8000.75


In [154]:
conn.close()